In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import cftime
import datetime
from datetime import date
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import font_manager
from matplotlib.cm import ScalarMappable
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
import numpy
import pandas
import xarray as xr

In [2]:
# Define Directories
Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Publish/Storm_Data_Raw/'
Output_Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Publish/Storm_Tracking/'

In [3]:
# Open File
def Create_DF(File):
    Data = open(File, 'r')
    Rows = []
#
# Organize Data From Raw Dataset
    for Line in Data:
        Rows.append(Line.strip())
    Storm_Code, Storm_List_Orig = [], []
    for i in range(len(Rows)):
        if Rows[i][0:5] == 'start':
            Code = Rows[i][41:45]
            Storm_List_Orig.append(Code)
        else:
            Storm_Code.append(Code)
    Array = numpy.zeros((13, len(Rows)-len(Storm_List_Orig)))
    Time = []
    k = -1
    for i in range(len(Rows)):
        if Rows[i][0:5] == 'start':
            k += 1
        else:
            l = len(Rows[i]) - 100
            Array[0][i-k-1] = float(Rows[i][0:6+l])
            Array[1][i-k-1] = float(Rows[i][8+l:14+l])
            Array[2][i-k-1] = float(Rows[i][17+l:24+l])
            Array[3][i-k-1] = float(Rows[i][26+l:31+l])
            Array[4][i-k-1] = float(Rows[i][34+l:41+l])
            Array[5][i-k-1] = float(Rows[i][44+l:51+l])
            Array[6][i-k-1] = float(Rows[i][54+l:61+l])
            Array[7][i-k-1] = float(Rows[i][64+l:71+l])
            Array[8][i-k-1] = float(Rows[i][74+l:81+l])
            Time.append(datetime.datetime(year=int(Rows[i][84+l:88+l]), month=int(Rows[i][90+l:92+l]), \
            day=int(Rows[i][94+l:96+l]), hour=int(Rows[i][98+l:100+l])))
#
# Create DataFrame to Store Data
    DF_All = pandas.DataFrame({"Orig Code": Storm_Code, "Lon": Array[0], "Lat": Array[1], "SLP(hPa)": Array[2], \
    "Winds(m/s)": Array[3], "Dist(m)": Array[4], "Angle": Array[5], "B": Array[6], "VLT": Array[7], "VUT": Array[8], \
    "Orig Time(Z)": Time})
#
# Remove Data Points Missing Phase Space Parameter Values (-999.0)
    DF = DF_All[(DF_All["B"] > -728) & (DF_All["VLT"] > -728) & (DF_All["VUT"] > -728)].reset_index()
    return (DF, Storm_List_Orig)

In [4]:
# Combine The Three Datasets Into One DataFrame of 90 or 93 Years For Each Scenario
def Combine_DF(DF_A, DF_B, DF_C, Orig_List_A, Orig_List_B, Orig_List_C, Model):
# Change Date For Simplicity of Analysis
# Control Assigned to Years in the 20th Century
# Each Year Within the Century are Independent
    if Model == "Control":
        Year_Start = 1900
# RCP4.5 Assigned to Years in the 21st Century
    elif Model == "RCP45":
        Year_Start = 2000
# RCP8.5 Assigned to Years in the 22nd Century
    elif Model == "RCP85":
        Year_Start = 2100
# Create Empty Lists
    Origi_List_A, Origi_List_B, Origi_List_C = [], [], []
    New_List_A, New_List_B, New_List_C = [], [], []
    New_Time, New_Code, ABC = [], [], []
    Orig_Combine, New_Combine = [], []
# Dataset A
    Year = Year_Start
    Orig_Year = DF_A["Orig Time(Z)"][0].year
    Year_Diff_A = Year - Orig_Year
    Count = 0
    for k in range(len(Orig_List_A)):
        try:
            Code, Year, Count = Update_Code(DF_A, Orig_List_A[k], Year_Diff_A, Year, Count)
            Origi_List_A.append(Orig_List_A[k])
            New_List_A.append(Code)
        except:
            Except = 0
# Dataset B
    Year += 1
    Orig_Year = DF_B["Orig Time(Z)"][0].year
    Year_Diff_B = Year - Orig_Year
    Count = 0
    for k in range(len(Orig_List_B)):
        try:
            Code, Year, Count = Update_Code(DF_B, Orig_List_B[k], Year_Diff_B, Year, Count)
            Origi_List_B.append(Orig_List_B[k])
            New_List_B.append(Code)
        except:
            Except = 0
# Dataset C
    Year += 1
    Orig_Year = DF_C["Orig Time(Z)"][0].year
    Year_Diff_C = Year - Orig_Year
    Count = 0
    for k in range(len(Orig_List_C)):
        try:
            Code, Year, Count = Update_Code(DF_C, Orig_List_C[k], Year_Diff_C, Year, Count)
            Origi_List_C.append(Orig_List_C[k])
            New_List_C.append(Code)
        except:
            Except = 0
#
# Combine Into New Time and Codes List
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_A, Origi_List_A, New_List_A, Year_Diff_A, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "A")
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_B, Origi_List_B, New_List_B, Year_Diff_B, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "B")
    New_Code, New_Time, ABC, Orig_Combine, New_Combine = Combine_Lists(DF_C, Origi_List_C, New_List_C, Year_Diff_C, \
    New_Code, New_Time, ABC, Orig_Combine, New_Combine, "C")
#
# Concatenate Other Variables
    Vars = ["Lon", "Lat", "SLP(hPa)", "Winds(m/s)", "Dist(m)", "Angle", "B", "VLT", "VUT"]
    Total_Length = len(New_Code)
    Array = numpy.zeros((9,Total_Length))
    for n in range(len(Vars)):
        List = []
        for k in range(len(DF_A)):
            List.append(DF_A[Vars[n]][k])
        for k in range(len(DF_B)):
            List.append(DF_B[Vars[n]][k])
        for k in range(len(DF_C)):
            List.append(DF_C[Vars[n]][k])
        Array[n] = List
#
# Create DataFrame to Store Data
    DF_Combine = pandas.DataFrame({"Code": New_Code, "Lon": Array[0], "Lat": Array[1], "SLP(hPa)": Array[2], \
    "Winds(m/s)": Array[3], "B": Array[6], "VLT": Array[7], "VUT": Array[8], "Time(Z)": New_Time})
    Codes_DF = pandas.DataFrame({"ABC": ABC, "Orig Code": Orig_Combine, "New Code": New_Combine})
    Year_Diffs = numpy.array([Year_Diff_A, Year_Diff_B, Year_Diff_C])
    return (DF_Combine, Codes_DF, Year_Diffs)

In [5]:
# Update Storm Code to Contain Year and Storm Number
def Update_Code(DF, Orig_Code, Year_Diff_A, Year, Count):
    DF_Storm = DF[DF["Orig Code"] == Orig_Code].reset_index()
    Orig_Year = DF_Storm["Orig Time(Z)"][0].year
    Time_Year = Orig_Year + Year_Diff_A
    if Time_Year == Year:
        Count += 1
    else:
        Year += 1
        Count = 1
    if Count < 10:
        Code = "TC"+str(Year)+"0"+str(Count)
    else:
        Code = "TC"+str(Year)+str(Count)
    return (Code, Year, Count)

In [6]:
# Combine Lists
def Combine_Lists(DF, Orig_List, New_List, Year_Diff, New_Code, New_Time, ABC, Orig_Combine, New_Combine, Y):
    for i in range(len(DF)):
        for l in range(len(Orig_List)):
# Update Storm Code
            if DF["Orig Code"][i] == Orig_List[l]:
                New_Code.append(New_List[l])
# Update Year
                Orig_Time = DF["Orig Time(Z)"][i]
                New_Time.append(Update_Year(Orig_Time, Year_Diff))
    for m in range(len(Orig_List)):
        ABC.append(Y)
        Orig_Combine.append(Orig_List[m])
        New_Combine.append(New_List[m])
    return (New_Code, New_Time, ABC, Orig_Combine, New_Combine)

In [7]:
# Change Year of Data
def Update_Year(Orig_Time, Year_Diff):
    Year_Update = Orig_Time.year + Year_Diff
# Febuary 29 Issues
    if Orig_Time.month == 2 and Orig_Time.day == 29:
        New_Time = Orig_Time.replace(year=Year_Update, day=28)
    else:
        New_Time = Orig_Time.replace(year=Year_Update)
    return (New_Time)

In [8]:
# Open Name List File
def Open_Name_List(File):
    Name_File = pandas.read_csv(File)
    Headings = list(Name_File.columns)
    Name_List = []
    for i in range(len(Headings)):
        for j in range(len(Name_File)):
            Name_List.append(Name_File[Headings[i]][j])
    return (Name_List)

In [9]:
# Assign Names to Each Cyclone Code
def Assign_Name(Name_List, Codes, DF):
    Storm_Names = []
    for i in range(len(Codes)):
        Name = Name_List[int(i%len(Name_List))]
        Storm_Names.append(Name)
    Codes["Name"] = Storm_Names
    DF_Names = []
    for k in range(len(DF)):
        Code = DF["Code"][k]
        Name = list(Codes[Codes["New Code"] == Code]["Name"])[0]
        DF_Names.append(Name)
    DF.insert(1, "Name", DF_Names)
    return (Codes, DF)

In [10]:
# Function to Find Distance Between Two Points
def Find_Distance(y1, y2, x1, x2):
    Start_Lat = y1 * numpy.pi / 180
    End_Lat = y2 * numpy.pi / 180
    Start_Lon = x1 * numpy.pi / 180
    End_Lon = x2 * numpy.pi / 180
    Lat_Diff = End_Lat - Start_Lat
    Lon_Diff = End_Lon - Start_Lon
    Earth_Rad = 6378
    Distance = 2 * Earth_Rad * numpy.sqrt((numpy.sin(Lat_Diff/2))**2 + \
    numpy.cos(Start_Lat) * numpy.cos(End_Lat) * (numpy.sin(Lon_Diff/2))**2)
    return (Distance)

In [11]:
# Find a Specific Storm Within the DataFrame
def Find_Storm(DF, Code):
    DF_Storm = DF[DF["Code"] == Code].reset_index()
    return (DF_Storm)

In [12]:
# Open Name List
Name_List = Open_Name_List('Storm_Tracking/Storm_Name_List.csv')

In [13]:
# Apply Functions to Open Datasets
Control_A_Data_V0, Control_A_Storm_List_V0 = Create_DF(Diri + 'traj_et_dtime900.control.002_avg')
Control_B_Data_V0, Control_B_Storm_List_V0 = Create_DF(Diri + 'traj_et_dtime900.control.003_avg')
Control_C_Data_V0, Control_C_Storm_List_V0 = Create_DF(Diri + 'traj_et_dtime900.control_avg')

In [14]:
# Process Control DataFrames
Control_Data_V1, Control_Codes_V1, Control_Year_Diffs = \
Combine_DF(Control_A_Data_V0, Control_B_Data_V0, Control_C_Data_V0, \
Control_A_Storm_List_V0, Control_B_Storm_List_V0, Control_C_Storm_List_V0, "Control")
Control_Codes_V2, Control_Data_V2 = Assign_Name(Name_List, Control_Codes_V1, Control_Data_V1)

In [15]:
# Apply Functions to Open Datasets
RCP45_A_Data_V0, RCP45_A_Storm_List_V0 = Create_DF(Diri + 'traj_et_dtime900.rcp45_avg')
RCP45_B_Data_V0, RCP45_B_Storm_List_V0 = Create_DF(Diri + 'traj_et_dtime900.rcp45.002_avg')
RCP45_C_Data_V0, RCP45_C_Storm_List_V0 = Create_DF(Diri + 'traj_et_dtime900.rcp45.003_avg')

In [16]:
# Process RCP4.5 DataFrames
RCP45_Data_V1, RCP45_Codes_V1, RCP45_Year_Diffs = \
Combine_DF(RCP45_A_Data_V0, RCP45_B_Data_V0, RCP45_C_Data_V0, \
RCP45_A_Storm_List_V0, RCP45_B_Storm_List_V0, RCP45_C_Storm_List_V0, "RCP45")
RCP45_Codes_V2, RCP45_Data_V2 = Assign_Name(Name_List, RCP45_Codes_V1, RCP45_Data_V1)

In [17]:
# Apply Functions to Open Datasets
RCP85_A_Data_V0, RCP85_A_Storm_List_V0 = Create_DF(Diri + 'traj_et_dtime900.rcp85_avg')
RCP85_B_Data_V0, RCP85_B_Storm_List_V0 = Create_DF(Diri + 'traj_et_dtime900.rcp85.003_avg')
RCP85_C_Data_V0, RCP85_C_Storm_List_V0 = Create_DF(Diri + 'traj_et_dtime900.rcp85.004_avg')

In [18]:
# Process RCP8.5 DataFrames
RCP85_Data_V1, RCP85_Codes_V1, RCP85_Year_Diffs = \
Combine_DF(RCP85_A_Data_V0, RCP85_B_Data_V0, RCP85_C_Data_V0, \
RCP85_A_Storm_List_V0, RCP85_B_Storm_List_V0, RCP85_C_Storm_List_V0, "RCP85")
RCP85_Codes_V2, RCP85_Data_V2 = Assign_Name(Name_List, RCP85_Codes_V1, RCP85_Data_V1)

In [19]:
# Check For Repeated Entries
def Repeated_Check(DF):
    for i in range(len(DF)):
        Lat = DF["Lat"][i]
        Lon = DF["Lon"][i]
        Time = DF["Time(Z)"][i]
        DF_Repeat = DF[(DF["Lat"] == Lat) & (DF["Lon"] == Lon) & (DF["Time(Z)"] == Time)]
# Deleted Repeated Entries
        if len(DF_Repeat) > 1:
            print (DF["Code"][i], Time)

In [20]:
Repeated_Check(Control_Data_V1)

In [21]:
# Function For Check For Time Discontinuity
def Time_Discontinuity(DF_Storm):
    Time_Continuous = 0
    for k in range(len(DF_Storm)):
        if k > 0 and Time_Continuous == 0:
#            print (k)
# If Over 24 Hours of Missing Data
            if DF_Storm["Time(Z)"][k] - DF_Storm["Time(Z)"][k-1] >= datetime.timedelta(hours=24):
# Divide Array Into Before The Time Discontinuity and After It
                Before = DF_Storm["Time(Z)"][k-1]
                After = DF_Storm["Time(Z)"][k]
                DF_Before = DF_Storm[DF_Storm["Time(Z)"] <= Before].reset_index()
                DF_After = DF_Storm[DF_Storm["Time(Z)"] >= After].reset_index()
# Choose the Part with the Lowest Min SLP to Keep
                if numpy.min(DF_After["SLP(hPa)"]) <= numpy.min(DF_Before["SLP(hPa)"]) and \
                len(DF_After) > len(DF_Before):
                    DF_Storm_Updated = DF_After
                else:
                    DF_Storm_Updated = DF_Before
                Time_Continuous = 1
                DF_Storm_Updated = DF_Storm_Updated.drop("level_0", axis=1)
# Repeat Until No More Time Discontinuity
    if Time_Continuous == 0:
        DF_Storm_Updated = DF_Storm.copy()
    return (DF_Storm_Updated)

In [22]:
# Function For Check For Track Discontinuity
def Track_Discontinuity(DF_Storm):
    Track_Continuous = 0
    Track_Discon = []
    for k in range(len(DF_Storm)):
        if k > 1 and Track_Continuous == 0:
# Calculate Distance Between Each 6 Hour Interval
            Distance_1 = Find_Distance(DF_Storm["Lat"][k], DF_Storm["Lat"][k-1], \
            DF_Storm["Lon"][k], DF_Storm["Lon"][k-1])
            Distance_2 = Find_Distance(DF_Storm["Lat"][k-1], DF_Storm["Lat"][k-2], \
            DF_Storm["Lon"][k-1], DF_Storm["Lon"][k-2])
# If Distance >= 800km and Distance Over 5 Times Further Than Previous Time Interval
            if Distance_1 >= 800 and Distance_1 >= Distance_2 * 5:
                Track_Discon = [DF_Storm["Code"][0], DF_Storm["Time(Z)"][k]]
# Divide Array Into Before The Track Discontinuity and After It
                Before = DF_Storm["Time(Z)"][k-1]
                After = DF_Storm["Time(Z)"][k]
                DF_Before = DF_Storm[DF_Storm["Time(Z)"] <= Before].reset_index()
                DF_After = DF_Storm[DF_Storm["Time(Z)"] >= After].reset_index()
# Choose the Part with the Lowest Min SLP to Keep
                if numpy.min(DF_After["SLP(hPa)"]) <= numpy.min(DF_Before["SLP(hPa)"]) and \
                len(DF_After) > len(DF_Before):
                    DF_Storm_Updated = DF_After
                else:
                    DF_Storm_Updated = DF_Before
                Track_Continuous = 1
                DF_Storm_Updated = DF_Storm_Updated.drop("level_0", axis=1)
# Repeat Until No More Track Discontinuity
    if Track_Continuous == 0:
        DF_Storm_Updated = DF_Storm.copy()
    return (DF_Storm_Updated, Track_Discon)

In [23]:
# Fix Storms That Have Time Discontinuity or Track Discontinuity
def DF_Fixing(DF_Storm_Orig, Track_Discons):
    Bottom_Right = False
    Top_Right = False
    Bottom_Left = False
    Top_Left = False
    ET_Complete = False
#
# Check For Time Discontinuity
    DF_Storm = Time_Discontinuity(DF_Storm_Orig)
# Repeat to Check Again
    DF_Storm = Time_Discontinuity(DF_Storm)
    DF_Storm = Time_Discontinuity(DF_Storm)
#
# Check For Track Discontinuity
    DF_Storm, Track_Discon = Track_Discontinuity(DF_Storm)
    if len(Track_Discon) > 0:
        Track_Discons.append(Track_Discon)
# Repeat to Check Again
    DF_Storm, Track_Discon = Track_Discontinuity(DF_Storm)
    if len(Track_Discon) > 0:
        Track_Discons.append(Track_Discon)
    DF_Storm, Track_Discon = Track_Discontinuity(DF_Storm)
    if len(Track_Discon) > 0:
        Track_Discons.append(Track_Discon)
    return (DF_Storm, Track_Discons)

In [24]:
# Function to Determine Whether a Storm Has Completed ET, and Find Begin and Complete Time of ET
def ET_Function(DF_Storm):
    Tropical_Begin = False
    ET_Begin = False
    ET_Complete = False
    Bottom_Right_t, Top_Right_t, Bottom_Left_t = [], [], []
    ET_Begin_t, ET_Compl_t, Trans_Type = -728, -728, -728
    for t in range(len(DF_Storm)):
        if ET_Complete == False:
            if Tropical_Begin == False:
# Storm Must First Be Tropical
                if DF_Storm["B"][t] <= 15 and DF_Storm["VLT"][t] >= 0:
                    Tropical_Begin = True
                    Bottom_Right_t.append(t)
            if Tropical_Begin == True:
# Storm Must Be Tropical For At Least 48 Hours
                if len(Bottom_Right_t) <= 8:
                    if DF_Storm["B"][t] <= 15 and DF_Storm["VLT"][t] >= 0:
                        Bottom_Right_t.append(t)
#
# Enter Next Stage If Storm Remains Tropical For At Least 48 Hours
                else:
                    if ET_Begin == False:
# If Storm Enters Top Right Quadrant = ET Begin
                        if DF_Storm["B"][t] > 15 and DF_Storm["VLT"][t] >= 0:
                            ET_Begin = True
                            ET_Begin_t = t
                            Top_Right_t.append(t)
# If Storm Enters Bottom Left Quadrant = ET Begin
                        elif DF_Storm["B"][t] <= 15 and DF_Storm["VLT"][t] < 0:
                            ET_Begin = True
                            ET_Begin_t = t
                            Bottom_Left_t.append(t)
# If Storm Directly Enters Top Left Quadrant = ET Complete
                        elif DF_Storm["B"][t] > 15 and DF_Storm["VLT"][t] < 0:
                            ET_Complete = True
                            ET_Begin_t, ET_Compl_t = t, t
                            Trans_Type = 3
#
# After ET Initiation
                    elif ET_Begin == True:
                        if len(Top_Right_t) > 0 and len(Top_Right_t) >= len(Bottom_Left_t):
# If Storm Remains in Top Right Quadrant
                            if DF_Storm["B"][t] > 15 and DF_Storm["VLT"][t] >= 0:
                                Top_Right_t.append(t)
# If Storm Enters Top Left Quadrant = ET Complete
                            elif DF_Storm["VLT"][t] < 0:
                                if DF_Storm["B"][t] > 10:
                                    ET_Complete = True
                                    ET_Compl_t, Trans_Type = t, 1
# If Storm Enters Bottom Left Quadrant
                                else:
                                    Bottom_Left_t.append(t)
                            elif DF_Storm["B"][t] <= 15 and DF_Storm["VLT"][t] >= 0:
# If Storm Returns to Bottom Right Quadrant For Over 24 Hours
                                if t-4 > Top_Right_t[len(Top_Right_t)-1]:
                                    ET_Begin = False
                                    Top_Right_t = []
                        elif len(Bottom_Left_t) > 0:
# If Storm Remains in Bottom Left Quadrant
                            if DF_Storm["B"][t] <= 15 and DF_Storm["VLT"][t] < 0:
                                Bottom_Left_t.append(t)
# If Storm Enters Top Left Quadrant = ET Complete
                            elif DF_Storm["B"][t] > 15:
                                if DF_Storm["VLT"][t] <= 0:
                                    ET_Complete = True
                                    ET_Compl_t, Trans_Type = t, 2
# If Storm Enters Top Right Quadrant
                                else:
                                    Top_Right_t.append(t)
                            elif DF_Storm["B"][t] <= 15 and DF_Storm["VLT"][t] >= 0:
# If Storm Returns to Bottom Right Quadrant For Over 24 Hours
                                if t-4 > Bottom_Left_t[len(Bottom_Left_t)-1]:
                                    ET_Begin = False
                                    Bottom_Left_t = []
#
# If Storm Did Not Spend At Least 48 Hours as Tropical
    if len(Bottom_Right_t) <= 8:
        Trans_Type = -728
# If ET Never Begun:
    elif ET_Compl_t == -728:
        if ET_Begin_t == -728:
            Trans_Type = -1
# If ET Began But Did Not Complete:
        else:
            Trans_Type = 0
    return (Trans_Type, ET_Begin_t, ET_Compl_t)

In [25]:
# Create DF With Extratropical Transition Information of Each Storm
def Storm_Info_DF(DF, Codes):
    ABC = []
    Trans_Types, Begin_Times, Compl_Times, Track_Discons = [], [], [], []
    for i in range(len(Codes)):
        DF_Storm_Orig = DF[DF["Code"] == Codes["New Code"][i]].reset_index()
# Apply Functions
        DF_Storm, Track_Discons = DF_Fixing(DF_Storm_Orig, Track_Discons)
        Trans_Type, ET_Begin_t, ET_Compl_t = ET_Function(DF_Storm)
        if ET_Begin_t > 0:
            Begin_Time = DF_Storm["Time(Z)"][ET_Begin_t]
        else:
            Begin_Time = numpy.nan
        if ET_Compl_t > 0:
            Compl_Time = DF_Storm["Time(Z)"][ET_Compl_t]
        else:
            Compl_Time = numpy.nan
        Begin_Times.append(Begin_Time)
        Compl_Times.append(Compl_Time)
        Trans_Types.append(Trans_Type)
        ABC.append(Codes["ABC"][i])
        if i == 0:
            DF_Fixed = DF_Storm.copy()
        else:
            DF_Fixed = pandas.concat([DF_Fixed, DF_Storm])
    DF_Fixed = DF_Fixed.reset_index()
    DF_Fixed = DF_Fixed.drop("index", axis=1)
#
# Create Storms DF
    Storms_DF_Orig = pandas.DataFrame({"Code": Codes["New Code"], "Name": Codes["Name"], \
    "ABC": ABC, "Trans Type": Trans_Types, "ET Begin Time": Begin_Times, "ET Complete Time": Compl_Times})
    return (DF_Fixed, Storms_DF_Orig)

In [26]:
# Apply Function For Control DataFrames
Control_Data_V3, Control_ET_V1 = Storm_Info_DF(Control_Data_V2, Control_Codes_V2)

In [27]:
# Apply Function For RCP4.5 DataFrames
RCP45_Data_V3, RCP45_ET_V1 = Storm_Info_DF(RCP45_Data_V2, RCP45_Codes_V2)

In [28]:
# Apply Function For RCP8.5 DataFrames
RCP85_Data_V3, RCP85_ET_V1 = Storm_Info_DF(RCP85_Data_V2, RCP85_Codes_V2)

In [29]:
# Add Storm Phase into DataFrame
def Storm_Phase_Info(Data_DF_Orig, ET_DF):
    Data_DF = Data_DF_Orig.copy()
    Storm_Phase_List = []
    for i in range(len(ET_DF)):
        DF_Storm = Data_DF[Data_DF["Code"] == ET_DF["Code"][i]].reset_index()
        Trans_Type, Begin_Time, Compl_Time = ET_DF["Trans Type"][i], ET_DF["ET Begin Time"][i], ET_DF["ET Complete Time"][i]
# Find Storm Phase Based on ET Begin Time and ET Complete Time
        Storm_Phase = Find_Storm_Phase(DF_Storm, Begin_Time, Compl_Time, Trans_Type)
        for j in range(len(Storm_Phase)):
            Storm_Phase_List.append(Storm_Phase[j])
    Data_DF["Storm Phase"] = Storm_Phase_List
    Data_DF = Data_DF.drop("level_0", axis=1)
    return (Data_DF)

In [30]:
# Function to Find Storm Phase
def Find_Storm_Phase(DF_Storm, Begin_Time, Compl_Time, Trans_Type):
    Time = list(DF_Storm["Time(Z)"])
    Storm_Phase = []
    for k in range(len(DF_Storm)):
# If Before ET Begin = Tropical
# If Between ET Begin and ET Complete = Transitioning
# If After ET Complete = Extratropical
        if Trans_Type <= -1:
            Storm_Phase.append("Tropical")
        elif Trans_Type == 0:
            if Time[k] < Begin_Time:
                Storm_Phase.append("Tropical")
            else:
                Storm_Phase.append("Transition")
        else:
            if Time[k] < Begin_Time:
                Storm_Phase.append("Tropical")
            elif Time[k] < Compl_Time:
                Storm_Phase.append("Transition")
            else:
                Storm_Phase.append("Extratropical")
    return (Storm_Phase)

In [31]:
# Apply Function
Control_Data_V4 = Storm_Phase_Info(Control_Data_V3, Control_ET_V1)
RCP45_Data_V4 = Storm_Phase_Info(RCP45_Data_V3, RCP45_ET_V1)
RCP85_Data_V4 = Storm_Phase_Info(RCP85_Data_V3, RCP85_ET_V1)

In [32]:
# Add Lat, Lon, SLP Into ET DataFrame
def ET_Variables(Main_DF, ET_DF_Orig):
    ET_DF = ET_DF_Orig.copy()
    Array = numpy.zeros((15, len(ET_DF)))
    Trop_Peak_Times = []
    Peak_Times = []
    Genesis_Times = []
    for i in range(len(ET_DF)):
        Code = ET_DF["Code"][i]
        Trans_Type, Begin_Time, Compl_Time = ET_DF["Trans Type"][i], ET_DF["ET Begin Time"][i], ET_DF["ET Complete Time"][i]
        Storm = Main_DF[Main_DF["Code"] == Code].reset_index()
#
# Find Lat, Lon, Time at Storm Peak
        Min_SLP = numpy.min(Storm["SLP(hPa)"])
        Storm_Peak = Storm[Storm["SLP(hPa)"] == Min_SLP]
        Array[0][i] = Min_SLP
        Array[1][i] = float(Storm_Peak["Lon"].iloc[0])
        Array[2][i] = float(Storm_Peak["Lat"].iloc[0])
        Peak_Time = Storm_Peak["Time(Z)"].iloc[0]
        Peak_Times.append(Peak_Time)
#
# Find Lat, Lon, Time at Tropical Phase Peak
        Storm_Trop = Storm[Storm["Storm Phase"] == "Tropical"]
        Trop_Min_SLP = numpy.min(Storm_Trop["SLP(hPa)"])
        Trop_Peak = Storm[Storm["SLP(hPa)"] == Trop_Min_SLP]
        Array[3][i] = Trop_Min_SLP
        Array[4][i] = float(Trop_Peak["Lon"].iloc[0])
        Array[5][i] = float(Trop_Peak["Lat"].iloc[0])
        Trop_Peak_Time = Trop_Peak["Time(Z)"].iloc[0]
        Trop_Peak_Times.append(Trop_Peak_Time)
#
# Find SLP, Lon, Lat at ET Begin
        if Trans_Type > -1:
            Begin = Storm[Storm["Time(Z)"] == Begin_Time]
            Array[6][i] = float(Begin["SLP(hPa)"].iloc[0])
            Array[7][i] = float(Begin["Lon"].iloc[0])
            Array[8][i] = float(Begin["Lat"].iloc[0])
        else:
            Array[6][i], Array[7][i], Array[8][i] = numpy.nan, numpy.nan, numpy.nan
#
# Find SLP, Lon, Lat at ET Complete
        if Trans_Type > 0:
            Complete = Storm[Storm["Time(Z)"] == Compl_Time]
            Array[9][i] = float(Complete["SLP(hPa)"].iloc[0])
            Array[10][i] = float(Complete["Lon"].iloc[0])
            Array[11][i] = float(Complete["Lat"].iloc[0])
        else:
            Array[9][i], Array[10][i], Array[11][i] = numpy.nan, numpy.nan, numpy.nan
#
# Find SLP, Lon, Lat at Genesis
        Genesis_Time = Storm["Time(Z)"].iloc[1]
        Genesis_Times.append(Genesis_Time)
        Array[12][i] = float(Storm["SLP(hPa)"].iloc[1])
        Array[13][i] = float(Storm["Lon"].iloc[1])
        Array[14][i] = float(Storm["Lat"].iloc[1])
#
# Append Into ET DF
    ET_DF["Peak Time"] = Peak_Times
    ET_DF["Trop Peak Time"] = Trop_Peak_Times
    ET_DF["Genesis Time"] = Genesis_Times
    ET_DF["Peak SLP"] = Array[0]
    ET_DF["Peak Lon"] = Array[1]
    ET_DF["Peak Lat"] = Array[2]
    ET_DF["Trop Peak SLP"] = Array[3]
    ET_DF["Trop Peak Lon"] = Array[4]
    ET_DF["Trop Peak Lat"] = Array[5]
    ET_DF["ET Begin SLP"] = Array[6]
    ET_DF["ET Begin Lon"] = Array[7]
    ET_DF["ET Begin Lat"] = Array[8]
    ET_DF["ET Complete SLP"] = Array[9]
    ET_DF["ET Complete Lon"] = Array[10]
    ET_DF["ET Complete Lat"] = Array[11]
    ET_DF["Genesis SLP"] = Array[12]
    ET_DF["Genesis Lon"] = Array[13]
    ET_DF["Genesis Lat"] = Array[14]
    return (ET_DF)

In [33]:
# Apply Function
Control_ET_V2 = ET_Variables(Control_Data_V4, Control_ET_V1)
RCP45_ET_V2 = ET_Variables(RCP45_Data_V4, RCP45_ET_V1)
RCP85_ET_V2 = ET_Variables(RCP85_Data_V4, RCP85_ET_V1)

In [34]:
# Raw Dataset
print (len(Control_ET_V2), len(RCP45_ET_V2), len(RCP85_ET_V2))

954 748 654


In [35]:
# Filter Out Storms That Have Not Been Tropical For At Least 48 Hours
def Tropical_Filter(Data_DF_Orig, ET_DF_Orig, Codes_DF_Orig):
    Filtered_ET_0 = ET_DF_Orig[ET_DF_Orig["Trans Type"] > -728].reset_index().drop("index", axis=1)
# Filter Out Storms With Peak SLP >= 1000hPa
    Filtered_ET = Filtered_ET_0[Filtered_ET_0["Peak SLP"] < 1000].reset_index().drop("index", axis=1)
    Code_List = Filtered_ET["Code"]
    Filtered_DF = Data_DF_Orig[Data_DF_Orig["Code"].isin(Code_List)].reset_index().drop("index", axis=1)
    Filtered_Codes = Codes_DF_Orig[Codes_DF_Orig["New Code"].isin(Code_List)].reset_index().drop("index", axis=1)
    return (Filtered_DF, Filtered_ET, Filtered_Codes)

In [36]:
# Apply Function
Control_Data_V5, Control_ET_V5, Control_Codes_V5 = Tropical_Filter(Control_Data_V4, Control_ET_V2, Control_Codes_V2)
RCP45_Data_V5, RCP45_ET_V5, RCP45_Codes_V5 = Tropical_Filter(RCP45_Data_V4, RCP45_ET_V2, RCP45_Codes_V2)
RCP85_Data_V5, RCP85_ET_V5, RCP85_Codes_V5 = Tropical_Filter(RCP85_Data_V4, RCP85_ET_V2, RCP85_Codes_V2)

In [37]:
# Dataset A
print (len(Control_ET_V5), len(RCP45_ET_V5), len(RCP85_ET_V5))

649 460 407


In [38]:
# Count Transition Type
Control_ET_Count = numpy.zeros(3)
RCP45_ET_Count = numpy.zeros(3)
RCP85_ET_Count = numpy.zeros(3)
Control_ET_Count[0] = len(Control_ET_V5[Control_ET_V5["Trans Type"] <= -1])
Control_ET_Count[1] = len(Control_ET_V5[Control_ET_V5["Trans Type"] == 0])
Control_ET_Count[2] = len(Control_ET_V5[Control_ET_V5["Trans Type"] >= 1])
RCP45_ET_Count[0] = len(RCP45_ET_V5[RCP45_ET_V5["Trans Type"] <= -1])
RCP45_ET_Count[1] = len(RCP45_ET_V5[RCP45_ET_V5["Trans Type"] == 0])
RCP45_ET_Count[2] = len(RCP45_ET_V5[RCP45_ET_V5["Trans Type"] >= 1])
RCP85_ET_Count[0] = len(RCP85_ET_V5[RCP85_ET_V5["Trans Type"] <= -1])
RCP85_ET_Count[1] = len(RCP85_ET_V5[RCP85_ET_V5["Trans Type"] == 0])
RCP85_ET_Count[2] = len(RCP85_ET_V5[RCP85_ET_V5["Trans Type"] >= 1])

In [39]:
# Calculate Percentages of ET Completion
Control_ET_Percents = Control_ET_Count / len(Control_ET_V5)
RCP45_ET_Percents = RCP45_ET_Count / len(RCP45_ET_V5)
RCP85_ET_Percents = RCP85_ET_Count / len(RCP85_ET_V5)
print (Control_ET_Percents)
print (RCP45_ET_Percents)
print (RCP85_ET_Percents)

[0.20801233 0.35747304 0.43451464]
[0.22173913 0.36521739 0.41304348]
[0.19410319 0.31941032 0.48648649]


In [40]:
# Show Percentages of ET Completion
ET_Percents = pandas.DataFrame({"Trans Type": ["No ET", "ET Incomplete", "ET Complete"], \
"Control": Control_ET_Percents, "RCP4.5": RCP45_ET_Percents, "RCP8.5": RCP85_ET_Percents})
ET_Percents

,Trans Type,Control,RCP4.5,RCP8.5
0,No ET,0.208012,0.221739,0.194103
1,ET Incomplete,0.357473,0.365217,0.319410
2,ET Complete,0.434515,0.413043,0.486486


In [41]:
# Filter For Storms That Completed ET Transition
def ET_Filter(Data_DF_Orig, ET_DF_Orig):
    ET_DF = ET_DF_Orig[ET_DF_Orig["Trans Type"] > 0].reset_index().drop("index", axis=1)
    Code_List = ET_DF["Code"]
    Filtered_DF = Data_DF_Orig[Data_DF_Orig["Code"].isin(Code_List)].reset_index().drop("index", axis=1)
    return (Filtered_DF, ET_DF)

In [42]:
# Apply Function
Control_Data_V6, Control_ET_V6 = ET_Filter(Control_Data_V5, Control_ET_V5)
RCP45_Data_V6, RCP45_ET_V6 = ET_Filter(RCP45_Data_V5, RCP45_ET_V5)
RCP85_Data_V6, RCP85_ET_V6 = ET_Filter(RCP85_Data_V5, RCP85_ET_V5)

In [43]:
# Dataset B
print (len(Control_ET_V6), len(RCP45_ET_V6), len(RCP85_ET_V6))

282 190 198


In [44]:
# Filter For Storms With Trop Peak SLP <= 990, ET Begin SLP <= 1008 and ET Complete SLP <= 1008
def SLP_Filter(Data_DF_Orig, ET_DF_Orig):
    ET_DF = ET_DF_Orig[(ET_DF_Orig["Trop Peak SLP"] <= 990) & (ET_DF_Orig["ET Begin SLP"] <= 1008) & \
    (ET_DF_Orig["ET Complete SLP"] <= 1008)].reset_index().drop("index", axis=1)
    Code_List = ET_DF["Code"]
    Filtered_DF = Data_DF_Orig[Data_DF_Orig["Code"].isin(Code_List)].reset_index().drop("index", axis=1)
    return (Filtered_DF, ET_DF)

In [45]:
# Apply Function
Control_Data_V7, Control_ET_V7 = SLP_Filter(Control_Data_V6, Control_ET_V6)
RCP45_Data_V7, RCP45_ET_V7 = SLP_Filter(RCP45_Data_V6, RCP45_ET_V6)
RCP85_Data_V7, RCP85_ET_V7 = SLP_Filter(RCP85_Data_V6, RCP85_ET_V6)

In [46]:
# Filter For Storms Based on ET Location
def ET_Loc_Filter(Data_DF_Orig, ET_DF_Orig):
# Filter Out ETs Near European Coast
    ET_DF_Filt = ET_DF_Orig[(ET_DF_Orig["ET Complete Lon"] < -12)]
# Filter Out ETs Deep Inland Within North America
    ET_DF_Filt = ET_DF_Filt[(ET_DF_Filt["ET Begin Lat"] < 31) | (ET_DF_Filt["ET Begin Lon"] > -83)]
    ET_DF_Filt = ET_DF_Filt[(ET_DF_Filt["ET Begin Lat"] < 34) | (ET_DF_Filt["ET Begin Lon"] > -81)]
    ET_DF_Filt = ET_DF_Filt[(ET_DF_Filt["ET Begin Lat"] < 36) | (ET_DF_Filt["ET Begin Lon"] > -78)]
    ET_DF_Filt = ET_DF_Filt[(ET_DF_Filt["ET Begin Lat"] < 40) | (ET_DF_Filt["ET Begin Lon"] > -76)]
    ET_DF_Filt = ET_DF_Filt[(ET_DF_Filt["ET Begin Lat"] < 45) | (ET_DF_Filt["ET Begin Lon"] > -72)]
    ET_DF_Filt = ET_DF_Filt[(ET_DF_Filt["ET Begin Lat"] < 49) | (ET_DF_Filt["ET Begin Lon"] > -68)]
    ET_DF_Filt = ET_DF_Filt[(ET_DF_Filt["ET Begin Lat"] < 52) | (ET_DF_Filt["ET Begin Lon"] > -62)]
    ET_DF_Filt = ET_DF_Filt[(ET_DF_Filt["ET Complete Lat"] < 31) | (ET_DF_Filt["ET Complete Lon"] > -83)]
    ET_DF_Filt = ET_DF_Filt[(ET_DF_Filt["ET Complete Lat"] < 34) | (ET_DF_Filt["ET Complete Lon"] > -81)]
    ET_DF_Filt = ET_DF_Filt[(ET_DF_Filt["ET Complete Lat"] < 36) | (ET_DF_Filt["ET Complete Lon"] > -78)]
    ET_DF_Filt = ET_DF_Filt[(ET_DF_Filt["ET Complete Lat"] < 40) | (ET_DF_Filt["ET Complete Lon"] > -76)]
    ET_DF_Filt = ET_DF_Filt[(ET_DF_Filt["ET Complete Lat"] < 45) | (ET_DF_Filt["ET Complete Lon"] > -72)]
    ET_DF_Filt = ET_DF_Filt[(ET_DF_Filt["ET Complete Lat"] < 49) | (ET_DF_Filt["ET Complete Lon"] > -68)]
    ET_DF_Filt = ET_DF_Filt[(ET_DF_Filt["ET Complete Lat"] < 52) | (ET_DF_Filt["ET Complete Lon"] > -62)]
    ET_DF = ET_DF_Filt.reset_index().drop("index", axis=1)
    Code_List = ET_DF["Code"]
    Filtered_DF = Data_DF_Orig[Data_DF_Orig["Code"].isin(Code_List)].reset_index().drop("index", axis=1)
    return (Filtered_DF, ET_DF)

In [47]:
# Apply Function
Control_Data_V8, Control_ET_V8 = ET_Loc_Filter(Control_Data_V7, Control_ET_V7)
RCP45_Data_V8, RCP45_ET_V8 = ET_Loc_Filter(RCP45_Data_V7, RCP45_ET_V7)
RCP85_Data_V8, RCP85_ET_V8 = ET_Loc_Filter(RCP85_Data_V7, RCP85_ET_V7)

In [48]:
# Filter For Storms Based on Movement Direction
def ET_Direction_Filter(Data_DF_Orig, ET_DF_Orig):
# Only Storms That Move Northeastward Between ET Begin and ET Complete
    ET_DF_Filt = ET_DF_Orig[(ET_DF_Orig["ET Begin Lat"] <= ET_DF_Orig["ET Complete Lat"]) \
    & (ET_DF_Orig["ET Begin Lon"] <= ET_DF_Orig["ET Complete Lon"])]
# For Storms Where ET Begin == ET Complete
    ET_DF_Unsure = ET_DF_Filt[(ET_DF_Filt["ET Begin Lat"] == ET_DF_Filt["ET Complete Lat"]) \
    & (ET_DF_Filt["ET Begin Lon"] == ET_DF_Filt["ET Complete Lon"])]
    Unsure_Codes = list(ET_DF_Unsure["Code"])
    Codes_Remove = []
# Check If Storm Move Northeastward Between Last Tropical Datapoint and ET Complete
    for i in range(len(Unsure_Codes)):
        DF_Storm = Find_Storm(Data_DF_Orig, Unsure_Codes[i])
        DF_Storm_TC = DF_Storm[DF_Storm["Storm Phase"] == "Tropical"].reset_index()
        DF_Storm_ExTC = DF_Storm[DF_Storm["Storm Phase"] == "Extratropical"].reset_index()
        if (DF_Storm_TC["Lat"][len(DF_Storm_TC)-1] > DF_Storm_ExTC["Lat"][0]) or \
        (DF_Storm_TC["Lon"][len(DF_Storm_TC)-1] > DF_Storm_ExTC["Lon"][0]):
            Codes_Remove.append(Unsure_Codes[i])
# Remove Filtered Out Storms
    ET_DF = ET_DF_Filt[~ET_DF_Filt["Code"].isin(Codes_Remove)].reset_index().drop("index", axis=1)
    Code_List = ET_DF["Code"]
    Filtered_DF = Data_DF_Orig[Data_DF_Orig["Code"].isin(Code_List)].reset_index().drop("index", axis=1)
    return (Filtered_DF, ET_DF)

In [49]:
# Apply Function
Control_Data_V9, Control_ET_V9 = ET_Direction_Filter(Control_Data_V8, Control_ET_V8)
RCP45_Data_V9, RCP45_ET_V9 = ET_Direction_Filter(RCP45_Data_V8, RCP45_ET_V8)
RCP85_Data_V9, RCP85_ET_V9 = ET_Direction_Filter(RCP85_Data_V8, RCP85_ET_V8)

In [50]:
# Filter For Storms Based on Phase Space Parameters
def Phase_Space_Filter(Data_DF_Orig, ET_DF_Orig):
    Unsure_Codes = ET_DF_Orig["Code"]
    Codes_Remove = []
    for i in range(len(Unsure_Codes)):
        DF_Storm = Find_Storm(Data_DF_Orig, Unsure_Codes[i])
        DF_Storm_TC = DF_Storm[DF_Storm["Storm Phase"] == "Tropical"].reset_index()
# Remove Storms With Trop Peak Outside Bottom Right Quadrant
        Trop_Peak_SLP = numpy.min(DF_Storm_TC["SLP(hPa)"])
        for j in range(len(DF_Storm_TC)):
            if DF_Storm_TC["SLP(hPa)"][j] == Trop_Peak_SLP:
                if (DF_Storm_TC["B"][j] > 15) or (DF_Storm_TC["VLT"][j] < 0):
                    if Unsure_Codes[i] not in Codes_Remove:
                        Codes_Remove.append(Unsure_Codes[i])
# Remove Filtered Out Storms
    ET_DF = ET_DF_Orig[~ET_DF_Orig["Code"].isin(Codes_Remove)].reset_index().drop("index", axis=1)
    Code_List = ET_DF["Code"]
    Filtered_DF = Data_DF_Orig[Data_DF_Orig["Code"].isin(Code_List)].reset_index().drop("index", axis=1)
#    print (Codes_Remove)
    return (Filtered_DF, ET_DF)

In [51]:
# Apply Function
Control_Data_V10, Control_ET_V10 = Phase_Space_Filter(Control_Data_V9, Control_ET_V9)
RCP45_Data_V10, RCP45_ET_V10 = Phase_Space_Filter(RCP45_Data_V9, RCP45_ET_V9)
RCP85_Data_V10, RCP85_ET_V10 = Phase_Space_Filter(RCP85_Data_V9, RCP85_ET_V9)

In [52]:
# Filter For Storms Based on Tropical Peak SLP and Tropical Duration
def Peak_SLP_Filter(Data_DF_Orig, ET_DF_Orig):
    Unsure_Codes = list(ET_DF_Orig["Code"])
    Codes_Remove = []
    for i in range(len(Unsure_Codes)):
        DF_Storm = Find_Storm(Data_DF_Orig, Unsure_Codes[i])
        DF_Storm_Trop = DF_Storm[DF_Storm["Storm Phase"] == "Tropical"].reset_index()
# Filter Out Storms With Genesis SLP - Tropical Peak SLP < 10:
        if DF_Storm_Trop["SLP(hPa)"][0] - numpy.min(DF_Storm_Trop["SLP(hPa)"]) < 10 and Unsure_Codes[i] not in Codes_Remove:
            Codes_Remove.append(Unsure_Codes[i])
# Filter Out Storms With Tropical Duration < 72 hours:
        elif len(DF_Storm_Trop["SLP(hPa)"]) < 12 and Unsure_Codes[i] not in Codes_Remove:
            Codes_Remove.append(Unsure_Codes[i])
# Remove Filtered Out Storms
    ET_DF = ET_DF_Orig[~ET_DF_Orig["Code"].isin(Codes_Remove)].reset_index().drop("index", axis=1)
    Code_List = ET_DF["Code"]
    Filtered_DF = Data_DF_Orig[Data_DF_Orig["Code"].isin(Code_List)].reset_index().drop("index", axis=1)
#    print (Codes_Remove)
    return (Filtered_DF, ET_DF)

In [53]:
# Apply Function
Control_Data_V11, Control_ET_V11 = Peak_SLP_Filter(Control_Data_V10, Control_ET_V10)
RCP45_Data_V11, RCP45_ET_V11 = Peak_SLP_Filter(RCP45_Data_V10, RCP45_ET_V10)
RCP85_Data_V11, RCP85_ET_V11 = Peak_SLP_Filter(RCP85_Data_V10, RCP85_ET_V10)

In [54]:
# Codes of Storm With Incomplete Composite Data
Incomplete_Codes = ["TC191013", "TC191309", "TC192010", "TC192211", "TC192306", "TC192904", "TC193601", "TC194802", "TC195107", "TC195309", "TC195808", \
"TC200004", "TC201309", "TC202406", "TC202508", "TC202601", "TC203303", "TC203803", "TC204304", "TC206201", "TC206601", "TC207801", "TC208806", \
"TC210005", "TC211501", "TC212201", "TC213506", "TC214601", "TC214805", "TC215101", "TC215702", "TC216202", "TC216209", "TC218204"]

In [55]:
# Filter Out Storms With Incomplete Composite Data
def Incomplete_Filter(Data_DF_Orig, ET_DF_Orig, Incomplete_Codes):
    Unsure_Codes = list(ET_DF_Orig["Code"])
    Codes_Remove = []
    for i in range(len(Unsure_Codes)):
        if Unsure_Codes[i] in Incomplete_Codes:
            Codes_Remove.append(Unsure_Codes[i])
    ET_DF = ET_DF_Orig[~ET_DF_Orig["Code"].isin(Codes_Remove)].reset_index().drop("index", axis=1)
    Code_List = ET_DF["Code"]
    Filtered_DF = Data_DF_Orig[Data_DF_Orig["Code"].isin(Code_List)].reset_index().drop("index", axis=1)
#    print (Codes_Remove)
    return (Filtered_DF, ET_DF)

In [56]:
# Apply Function
Control_Data_V12, Control_ET_V12 = Incomplete_Filter(Control_Data_V11, Control_ET_V11, Incomplete_Codes)
RCP45_Data_V12, RCP45_ET_V12 = Incomplete_Filter(RCP45_Data_V11, RCP45_ET_V11, Incomplete_Codes)
RCP85_Data_V12, RCP85_ET_V12 = Incomplete_Filter(RCP85_Data_V11, RCP85_ET_V11, Incomplete_Codes)

In [57]:
# Dataset C
print (len(Control_ET_V12), len(RCP45_ET_V12), len(RCP85_ET_V12))

89 48 55


In [58]:
# Table 1
# Show Number of Storms In Each Subset
Storm_Counts = pandas.DataFrame({"Dataset": ["Raw", "Dataset A", "Subset B", "Subset C"], \
"Control": [len(Control_ET_V2), len(Control_ET_V5), len(Control_ET_V6), len(Control_ET_V12)], \
"RCP4.5": [len(RCP45_ET_V2), len(RCP45_ET_V5), len(RCP45_ET_V6), len(RCP45_ET_V12)], \
"RCP8.5": [len(RCP85_ET_V2), len(RCP85_ET_V5), len(RCP85_ET_V6), len(RCP85_ET_V12)]})
Storm_Counts

,Dataset,Control,RCP4.5,RCP8.5
0,Raw,954,748,654
1,Dataset A,649,460,407
2,Subset B,282,190,198
3,Subset C,89,48,55


In [59]:
# Output DF to csv File
def Output_File(DF, Model, Type):
    File_Name = str(Model+'_'+Type+'_Output.csv')
    DF.to_csv(Output_Diri+File_Name)

In [60]:
# Output Control Files
Output_File(Control_Data_V5, "Control", "Data_DatasetA")
Output_File(Control_Data_V6, "Control", "Data_SubsetB")
Output_File(Control_Data_V12, "Control", "Data_SubsetC")
Output_File(Control_ET_V5, "Control", "ET_DatasetA")
Output_File(Control_ET_V6, "Control", "ET_SubsetB")
Output_File(Control_ET_V12, "Control", "ET_SubsetC")
Output_File(Control_Codes_V5, "Control", "Codes")

In [61]:
# Output RCP4.5 Files
Output_File(RCP45_Data_V5, "RCP45", "Data_DatasetA")
Output_File(RCP45_Data_V6, "RCP45", "Data_SubsetB")
Output_File(RCP45_Data_V12, "RCP45", "Data_SubsetC")
Output_File(RCP45_ET_V5, "RCP45", "ET_DatasetA")
Output_File(RCP45_ET_V6, "RCP45", "ET_SubsetB")
Output_File(RCP45_ET_V12, "RCP45", "ET_SubsetC")
Output_File(RCP45_Codes_V5, "RCP45", "Codes")

In [62]:
# Output RCP8.5 Files
Output_File(RCP85_Data_V5, "RCP85", "Data_DatasetA")
Output_File(RCP85_Data_V6, "RCP85", "Data_SubsetB")
Output_File(RCP85_Data_V12, "RCP85", "Data_SubsetC")
Output_File(RCP85_ET_V5, "RCP85", "ET_DatasetA")
Output_File(RCP85_ET_V6, "RCP85", "ET_SubsetB")
Output_File(RCP85_ET_V12, "RCP85", "ET_SubsetC")
Output_File(RCP85_Codes_V5, "RCP85", "Codes")